In [1]:
import cv2
import matplotlib.pyplot as plot
from skimage.feature import (match_descriptors,corner_harris,corner_peaks,ORB,plot_matches,BRIEF)
import os
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import pcl

In [ ]:
#%matplotlib inline

In [2]:
#matriz camara
# -----------------------------------------------------------------------> 
K = np.array([[2759.48/4, 0, 1520.69/4, 0, 2764.16/4,1006.81/4, 0, 0, 1]]).reshape(3, 3)
d = np.array([0.0, 0.0, 0.0, 0.0, 0.0]).reshape(1, 5)
# ----------------------------------------------------------------------->
#K = np.array([56.9105,0,11.8698,0,56.9105,8.0418,0,0,1]).reshape(3, 3) #original tomada del .CAM
#d = np.array([8.641e-006, 0.0, 0.0, 0.0, 0.0]).reshape(1, 5) #matriz de distorsion original tomada del .CAM
#---------------------------------------------------
#ruta = "img/candela/ZCH0001/"
#ruta = "img/candela/ZCH0131/"
ruta = "img/monumento/"

In [3]:
lstFiles = []
lstDir = os.walk(ruta)
for root, dirs, files in lstDir:
    for fichero in files:
        (nombreFichero, extension) = os.path.splitext(fichero)
        if(extension == ".png" or extension == ".JPG"):
            img = cv2.imread(ruta+nombreFichero+extension,cv2.CV_8UC3)
            #img = cv2.imread(ruta+nombreFichero+extension,1)
            if len(img.shape) == 2:
                img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
            #img = cv2.resize(img,(0,0),fx=0.5, fy=0.5)
            lstFiles.append(img)
            #print (nombreFichero+extension)

In [4]:
img1 = lstFiles[0]
img2 = lstFiles[1]

#redimencion
if img1.shape[1] > 600:
    while img1.shape[1] > 2*600:
        img1 = cv2.pyrDown(img1)
        img2 = cv2.pyrDown(img2)

img1 = cv2.undistort(img1, K, d)
img2 = cv2.undistort(img2, K, d)

#plot.imshow(img2)
#plot.show()

In [9]:
#ORB
surf = cv2.SURF()
first_key_points, first_desc = surf.detectAndCompute(img1,None)
second_key_points, second_desc = surf.detectAndCompute(img2,None)

# match descriptors
matcher = cv2.BFMatcher(cv2.NORM_L1, True)
matches = matcher.match(first_desc, second_desc)

# generate lists of point correspondences
first_match_points = np.zeros((len(matches), 2), dtype=np.float32)
second_match_points = np.zeros_like(first_match_points)

for i in range(len(matches)):
    if (matches[i].distance < 2):
        first_match_points[i] = first_key_points[matches[i].queryIdx].pt
        second_match_points[i] = second_key_points[matches[i].trainIdx].pt
    

In [37]:
#matches = sorted(matches, key = lambda x:x.distance)
#img4 = cv2.drawMatches(img1,first_key_points,img2,second_key_points,matches,None)
#plot.imshow(img4)


In [13]:
# matriz fundamental
retval,mask=cv2.findFundamentalMat(first_match_points,second_match_points,cv2.FM_RANSAC, 0.1, 0.99)
#Matriz esencial
E = K.T.dot(retval).dot(K)
retval

array([[ -1.34065634e-08,   1.21369110e-05,  -3.18495441e-03],
       [ -3.98740430e-06,   7.63708768e-07,  -2.77698772e-02],
       [  1.02166074e-03,   2.38064332e-02,   1.00000000e+00]])

In [14]:
#funcion que permite determinar si lo puntos se encuentra frente a la camara o no
def _in_front_of_both_cameras(first_points, second_points, rot, trans):
    rot_inv = rot
    for first, second in zip(first_points, second_points):
        first_z = np.dot(rot[0, :] - second[0]*rot[2, :],trans) / np.dot(rot[0, :] - second[0]*rot[2, :],second)
        first_3d_point = np.array([first[0] * first_z,second[0] * first_z, first_z])
        second_3d_point = np.dot(rot.T, first_3d_point) - np.dot(rot.T,trans)
        if first_3d_point[2] < 0 or second_3d_point[2] < 0:
            return False
    return True

In [15]:
#procedimiento dudoso

first_inliers = []
second_inliers = []
for i in range(len(mask)):
    if mask[i]:
        # normalize and homogenize the image coordinates
        first_inliers.append(np.linalg.inv(K).dot([first_match_points[i][0],first_match_points[i][1], 1.0]))
        second_inliers.append( np.linalg.inv(K).dot([second_match_points[i][0],second_match_points[i][1], 1.0]))

U,s,V= np.linalg.svd(E)
W = np.array([0.0, -1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0,1.0]).reshape(3, 3)
R = U.dot(W).dot(V) # U*W*V.T --> otra biblografia
T = U[:,2]


if not _in_front_of_both_cameras(first_inliers,second_inliers,R, T):
    # Second choice: R = U * W * Vt, T = -u_3
    T = - U[:, 2]
    print "entro0"

if not _in_front_of_both_cameras(first_inliers,second_inliers,R, T):
    # Third choice: R = U * Wt * Vt, T = u_3
    print "entro1"
    R = U.dot(W.T).dot(V)
    T = U[:, 2]
    if not _in_front_of_both_cameras(first_inliers,second_inliers, R, T):
        # Fourth choice: R = U * Wt * Vt, T = -u_3
        print "entro2"
        T = - U[:, 2]
        
first_inliers = np.array(first_inliers)
second_inliers = np.array(second_inliers)
R = U.dot(W.T).dot(V)
T = U[:, 2]

entro0
entro1
entro2


In [16]:
#validando matriz de rotacion ---> si el determinante es 1 o -1 todo OK.
np.linalg.det(R)
first_inliers.shape

(212, 3)

In [17]:
#P1 =np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0]]) #matriz de camara canonica! manera pro: 
P1 = np.hstack((np.eye(3), np.zeros((3, 1))))
P2 = np.hstack((R, T.reshape(3, 1))) # P2 = [R|T]
#P2 = np.array([[R[0][0],R[0][1],R[0][2],T[0]],[R[1][0],R[1][1],R[1][2],T[1]],[R[2][0],R[2][1],R[2][2],T[2]]]) #[R|t]

In [18]:
first_inliers1 = first_inliers.reshape(-1, 3)[:, :2]
second_inliers2 = second_inliers.reshape(-1,3)[:,:2]
triangulacion = cv2.triangulatePoints(P1,P2,first_inliers1.T,second_inliers2.T).T

In [19]:
#triangulacion
pts3D = triangulacion[:, :3]/np.repeat(triangulacion[:, 3], 3).reshape(-1, 3)

In [20]:
pts3D.shape

(212, 3)

In [21]:
def linearTriangulation(u1, P1, u2, P2):
        A = np.array([u1[0]*P1[2, 0] - P1[0, 0], u1[0]*P1[2, 1] - P1[0, 1],
                      u1[0]*P1[2, 2] - P1[0, 2], u1[1]*P1[2, 0] - P1[1, 0],
                      u1[1]*P1[2, 1] - P1[1, 1], u1[1]*P1[2, 2] - P1[1, 2],
                      u2[0]*P2[2, 0] - P2[0, 0], u2[0]*P2[2, 1] - P2[0, 1],
                      u2[0]*P2[2, 2] - P2[0, 2], u2[1]*P2[2, 0] - P2[1, 0],
                      u2[1]*P2[2, 1] - P2[1, 1],
                      u2[1]*P2[2, 2] - P2[1, 2]]).reshape(4, 3)

        B = np.array([-(u1[0]*P1[2, 3] - P1[0, 3]),
                      -(u1[1]*P1[2, 3] - P1[1, 3]),
                      -(u2[0]*P2[2, 3] - P2[0, 3]),
                      -(u2[1]*P2[2, 3] - P2[1, 3])]).reshape(4, 1)

        ret, X = cv2.solve(A, B, flags=cv2.DECOMP_SVD)
        return X.reshape(1, 3)

In [22]:
unos = np.ones((first_inliers1.shape[0],1))
u1 = np.hstack((first_inliers1,unos)) # normalizado a x,y,1
u2 = np.hstack((second_inliers2,unos)) # normalizado a x,y,1
X  = []
for i in range(first_inliers1.shape[0]):
    uf =  np.linalg.inv(K) * u1[i]
    us = np.linalg.inv(K) * u2[i]
    x = linearTriangulation(uf[0],P1,us[0],P2)
    X.append(x[0])
np.array(X).shape

cloud = pcl.PointCloud(np.array(np.concatenate((X,pts3D)), dtype=np.float32))

fil = cloud.make_statistical_outlier_filter()
fil.set_mean_k (50)
fil.set_std_dev_mul_thresh (1.0)
fil.filter().to_file("prueba.pcd")

0

In [23]:
# plot with matplotlib
Ys = pts3D[:, 0]
Zs = pts3D[:, 1]
Xs = pts3D[:, 2]
# ------------------------->
fig = plot.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(Xs, Ys, Zs, c='r', marker='o')
ax.set_xlabel('Y')
ax.set_ylabel('Z')
ax.set_zlabel('X')
ax.set_aspect('equal')
plot.show()
